#### Goal is to explore the functionality to generate phase response pulses using the mkidreadoutanalysis package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mkidreadoutanalysis.quasiparticletimestream import QuasiparticleTimeStream
from mkidreadoutanalysis.resonator import Resonator, RFElectronics, ReadoutPhotonResonator, FrequencyGrid, LineNoise

In [ ]:
# Like usual, lets start by creating the photons using the Quasiparticle time stream.
# We will modulate the quasiparticle density perturbations to get more realistic data
mags = np.arange(0, 1.1, 0.1)
sampling_rate = 1e6 # Hz
sampling_interval = 0.01 # Secs; number of samples = 1e6 * 1e-2 = 1e4 (10000)

qpt = QuasiparticleTimeStream(sampling_rate, sampling_interval)
qpt.gen_quasiparticle_pulse()
_ = qpt.gen_photon_arrivals(cps=1000)
_ = qpt.populate_photons()

In [ ]:
# Now lets visualize the quasiparticle density time stream
plt.figure(figsize=(10,5))
plt.plot(np.arange(qpt.data.size), qpt.data)
plt.xlabel('Time (us)')
plt.ylabel(r'$\Delta$ Quasiparticle Density')

In [ ]:
# With the quasiparticle density and photon timestreams generated, now lets create a resonator object
# that defines the resonator specific parameters, along with a representation of the readout electronics
# and frequency grid.
resonator = Resonator(f0=4.0012e9, # resonant frequency
                      qi=200000, # internal quality factor
                      qc=15000, # coupling quality factor
                      xa=1e-9, # asymmetry fractional detuning (See Nick's thesis, pg. 29)
                      a=0, # nonlinear saturation parameter (See Nick's thesis, pg. 30)
                      tls_scale=1 # Used to generate tls noise; should be the PSD of the noise at 1Hz
                      )
rf_electronics = RFElectronics(gain=(3.0, 0, 0), # Coeffecients for creating a quadratic, 1D polynomial representing gain function
                               phase_delay=0, # Loop rotation (radians)
                               cable_delay=50e-9 # Secs
                               )
# Frequency Grid object contains the frequency related "knobs" of the readout electronics (readout frequency range)
freq_grid = FrequencyGrid(fc=resonator.f0, # Center of the frequency sweep in Hz (set as the resonator's resonant freq.)
                          points=1000, # Resolution of the sweep
                          span=500e6 # Bandwidth of the sweep in Hz
                          )

In [ ]:
# With all the components created above, lets tie it all together to create an entire system.
# This will be without any added noise
readout = ReadoutPhotonResonator(resonator, qpt, freq_grid, rf_electronics, noise_on=False)

# Now that we have a readout system, let's generate the the phase response (without noise)
# using the original coordinate transformation (see Nick's thesis, pg. 32).
# This transformation has significantly lower resolving power at high energies.
phase_response, _ = readout.basic_coordinate_transformation()

In [ ]:
# Now lets plot the idealized phase response
plt.figure(figsize=(10,5))
plt.plot(np.arange(phase_response.size), phase_response)
plt.xlabel('Time (us)')
plt.ylabel('Phase Response (radians)')

In [ ]:
# Now lets add some noise to the system to make it more realistic
line_noise = LineNoise(freqs=[60, 50e3, 100e3, 250e3, -300e3, 300e3, 500e3], # Noise frequencies in Hz relative to FrequencyGrid center freq.
                       amplitudes=[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01], # Associated amplitudes of the noise frequencies above
                       phases=[0, 0.5, 0,1.3,0.5, 0.2, 2.4], # Associated phases
                       n_samples=100,
                       fs=1e2 # Sampling rate in Hz
                      )

noisy_electronics = RFElectronics((3,0,0),
                                  0,
                                  50e-9,
                                  white_noise_scale=30,
                                  line_noise=line_noise
                                  ) 

readout = ReadoutPhotonResonator(resonator, qpt, freq_grid, noisy_electronics, noise_on=True)

noisy_phase_response, _ = readout.basic_coordinate_transformation()
plt.figure(figsize=(10,5))
plt.plot(np.arange(noisy_phase_response.size), noisy_phase_response)
plt.xlabel('Time (us)')
plt.ylabel('Phase Response (radians)')